In [1]:
import pandas as pd
import numpy as np
import openpyxl

In [2]:
# Specify the path to your Excel file
excel_file_path = 'newCrib.xlsx'

# Load each sheet into separate data frames
df1 = pd.read_excel(excel_file_path, sheet_name='CDB_CDPU_CRIB_STATUS')
df3 = pd.read_excel(excel_file_path, sheet_name='CRIB_CREDIT_FAC_DTL')

In [3]:
df1

ID BOT_STATUS  REQ_NIC       HELP_ID CRIB_STATUS  FACILITY_AMOUNT  \
0  406677  COMPLETED       45  CRIB_0449877         HIT           200000   

   MARKETING_OFFICER_ID  ALERT_GSM  CRIB_SCORE  APPROVAL_STATUS  CREATED_BY  \
0               1010430  715516183        37.5              NaN         NaN   

   CREATED_DATE UPDATED_BY     UPDATED_DATE  VERSION  \
0           NaN     SYSTEM  00:31:29.700000        1   

                   CRIB_FILE_PATH  REQ_NIC.1  REQ_DATE  
0  comb_572100154V_1709202493.pdf        NaN  00:31:24

In [4]:
df3

ID  No       HELP_ID  REQ_NIC Running_NIC Inst_Catg Inst_Branch  \
0  314323   1  CRIB_0449877       45  572100154V       LFC         ---   

  CF_Type CF_Stat Own_Shp  ... Installment_Amount  Amount_Written_Off  \
0    CRCD    ACTV     GRT  ...                  0                   0   

          Reported_Date   First_Disburse_Date   Latest_Payment_Date  \
0  2024-01-31T00:00:00Z  2021-12-01T00:00:00Z  2023-12-31T00:00:00Z   

   Restruct_Date End_Date Repay_Type       Purp  Coverage  
0            NaN      NaN       DMND  07:03:001        FS  

[1 rows x 24 columns]

In [5]:
df3['newID'] = df3['Running_NIC'] + df3['No'].astype(str)
df3 = df3[['newID'] + [col for col in df3.columns if col != 'newID']]

In [6]:
df3.head(10)

newID      ID  No       HELP_ID  REQ_NIC Running_NIC Inst_Catg  \
0  572100154V1  314323   1  CRIB_0449877       45  572100154V       LFC   

  Inst_Branch CF_Type CF_Stat  ... Installment_Amount Amount_Written_Off  \
0         ---    CRCD    ACTV  ...                  0                  0   

          Reported_Date   First_Disburse_Date   Latest_Payment_Date  \
0  2024-01-31T00:00:00Z  2021-12-01T00:00:00Z  2023-12-31T00:00:00Z   

   Restruct_Date  End_Date Repay_Type       Purp Coverage  
0            NaN       NaN       DMND  07:03:001       FS  

[1 rows x 25 columns]

In [7]:
df3.columns

Index(['newID', 'ID', 'No', 'HELP_ID', 'REQ_NIC', 'Running_NIC', 'Inst_Catg',
       'Inst_Branch', 'CF_Type', 'CF_Stat', 'Own_Shp', 'Crcy',
       'Amount_Granted_Limit', 'Current_Balance', 'Arrears_Amount',
       'Installment_Amount', 'Amount_Written_Off', 'Reported_Date',
       'First_Disburse_Date', 'Latest_Payment_Date', 'Restruct_Date',
       'End_Date', 'Repay_Type', 'Purp', 'Coverage'],
      dtype='object')

In [8]:
df3['Arrears_Amount']

0    0
Name: Arrears_Amount, dtype: int64

In [9]:
import pandas as pd

# Create an empty list to store filtered rows
filtered_rows = []

def get_assigned_values( row):
    Arrears_Amount = row['Arrears_Amount']
    Installment_Amount = row['Installment_Amount']
    
    if Arrears_Amount > 0:
        CF_Type = row['CF_Type']
        if CF_Type in ['OVDR', 'CRCD']:
            Rental_Value = row['Arrears_Amount']
        else:
            Rental_Value = Installment_Amount
        row['Rental_Value'] = Rental_Value
        return row[['newID', 'Help_ID', 'CF_Type', 'CF_Stat', 'Arrears_Amount', 'Installment_Amount', 'Rental_Value']]
    else:
        # If Arrears_Amount is not greater than 0, return None
        return None

# Iterate through each row and append rows with Installment_Amount > 0 to the new list
for index, row in df3.iterrows():
    filtered_row = get_assigned_values(row)
    if filtered_row is not None:
        filtered_rows.append(filtered_row)




In [10]:
filtered_rows

[]

In [10]:
print(type(pd))
print(type(pd.DataFrame))

<class 'module'>
<class 'type'>


In [11]:
# Convert the list of filtered rows into a DataFrame
filtered_df = pd.DataFrame(filtered_rows, columns=['newID', 'Help_ID', 'CF_Type', 'CF_Stat', 'Arrears_Amount', 'Installment_Amount', 'Rental_Value'])

# Reset index for the new DataFrame
filtered_df.reset_index(drop=True, inplace=True)

# Display the new DataFrame
filtered_df

Empty DataFrame
Columns: [newID, Help_ID, CF_Type, CF_Stat, Arrears_Amount, Installment_Amount, Rental_Value]
Index: []

In [16]:
import pandas as pd

# Create an empty list to store modified rows
modified_rows = []

def calculate_arease_months(row):
    Arrears_Amount = row['Arrears_Amount']
    Rental_Value = row['Rental_Value']
    
    # Calculate Arease_Months
    if Rental_Value == 0:
        Arease_Months = 0
    else:
        Arease_Months = Arrears_Amount / Rental_Value
    
    # Append the row with added 'Arease_Months' column to the list
    row['Arease_Months'] = Arease_Months
    modified_rows.append(row)

# Iterate through each row and calculate 'Arease_Months' for each row
for index, row in filtered_df.iterrows():  
    calculate_arease_months(row)

# Convert the list of modified rows into a DataFrame
filtered_df = pd.DataFrame(modified_rows)

# Reset index for the new DataFrame
filtered_df.reset_index(drop=True, inplace=True)

# Display the new DataFrame
filtered_df


newID       Help_ID CF_Type CF_Stat  Arrears_Amount  \
0    813204592V5  CRIB_0452013    LOAN    ACTV          127719   
1    813204592V6  CRIB_0452013    LEAS    ACTV          354862   
2    813204592V7  CRIB_0452013    OVDR    ACTV           17013   
3    813204592V8  CRIB_0452013    LEAS    ACTV           90173   
4   813204592V10  CRIB_0452013    LEAS    ACTV           76794   
5   813204592V12  CRIB_0452013    LEAS    ACTV           61066   
6   813204592V13  CRIB_0452013    LOAN    ACTV            6501   
7   813204592V14  CRIB_0452013    LEAS    ACTV           67577   
8   813204592V15  CRIB_0452013    LOAN    MGCP           13357   
9   813204592V16  CRIB_0452013    LOAN    MGCP            9467   
10  813204592V17  CRIB_0452013    LEAS    LGAL          669267   

    Installment_Amount  Rental_Value  Arease_Months  
0                25000         25000       5.108760  
1                76374         76374       4.646372  
2                    0         17013       1.000000  
3                63964         63964       1.409746  
4               112425        112425       0.683069  
5                36271         36271       1.683604  
6                 8333          8333       0.780151  
7                30893         30893       2.187453  
8                 8335          8335       1.602519  
9                 8334          8334       1.135949  
10               78245         78245       8.553479

In [23]:
Tot_Arrears_Amount = filtered_df.Arrears_Amount.sum()
Tot_Rental_Value = filtered_df.Rental_Value.sum()

In [24]:
#score_value = (Tot_Rental_Value/ Tot_Arrears_Amount)
score_value = (Tot_Arrears_Amount/Tot_Rental_Value)
score_value

3.211173141123892

In [25]:
if 0 <= score_value <= 1:
    crib_sc = 1
elif 1 < score_value <= 2:
    crib_sc = 0.5
elif 2 < score_value <= 3:
    crib_sc = -0.5
elif 3 < score_value <= 6:
    crib_sc = -1
else:
    crib_sc = -2

print(f'Current month arrears status',crib_sc*0.1)



Current month arrears status -0.1


In [26]:
crib_sc

-1